In [ ]:
"""
NOTEBOOK 09: RESUMEN DE RESULTADOS PARA TESIS
==============================================
Recopilar, consolidar y presentar todos los análisis realizados
en formato submission-ready para la tesis doctoral.

Contenidos:
1. Figuras de calidad de publicación
2. Tablas de resultados estadísticos
3. Resumen de hallazgos principales
4. Metodología y validación
5. Limitaciones y conclusiones

Entrada: Todos los análisis previos (notebooks 01-08)
Salida: Figuras, tablas, resumen ejecutivo para tesis

Autor: Tesis Doctoral - Análisis Automatizado de Dinámicas Microvasculares
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

# Configuración de visualización - Publication quality
sns.set_style("whitegrid")
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

In [ ]:
# Save final summary report
report_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/THESIS_SUMMARY_REPORT.txt')
with open(report_path, 'w') as f:
    f.write(summary_stats)

print(f"Summary report saved to: {report_path}")

# Create a checklist of completed analyses
analysis_checklist = """
ANALYSIS PIPELINE COMPLETION CHECKLIST
=======================================

✓ Notebook 01: Data Exploration
  - Video metadata extraction
  - Frame quality assessment
  - Artifact identification and classification

✓ Notebook 02: Data Preprocessing  
  - Frame extraction and normalization
  - Video stabilization
  - Artifact mitigation

✓ Notebook 03: Annotation and Visualization
  - CSV data loading and validation
  - Vessel density analysis
  - Individual vessel velocity analysis
  - Temporal trend analysis

✓ Notebook 04: Vessel Segmentation
  - Multiple segmentation methods tested
  - Hybrid approach selected and optimized
  - Skeleton and morphological feature extraction

✓ Notebook 05: Segmented Video Analysis
  - Quantitative vessel metrics computation
  - Integration with manual annotations
  - Validation of segmentation quality

✓ Notebook 06: Space-Time Feature Extraction
  - Spatiotemporal feature computation
  - Velocity statistics and variability measures
  - Density-velocity relationship analysis

✓ Notebook 07: Microvascular Dynamics Modeling
  - Temporal trend analysis
  - Patient-specific dynamics models
  - Prediction of SVD/TVD changes

✓ Notebook 08: Clinical Metrics and Comparison
  - Clinical outcome classification
  - Statistical comparison between patient groups
  - Prognosis risk stratification

✓ Notebook 09: Results Summary for Thesis
  - Comprehensive statistical summary
  - Publication-ready visualizations
  - Key findings and recommendations

NEXT STEPS
==========
1. Review and validate all results
2. Fine-tune clinical classification thresholds if needed
3. Add more reference data or patient outcomes if available
4. Generate publication-ready figures
5. Correlate with clinical parameters (if available)
"""

print(analysis_checklist)

checklist_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/ANALYSIS_CHECKLIST.txt')
with open(checklist_path, 'w') as f:
    f.write(analysis_checklist)

print(f"\nAnalysis checklist saved to: {checklist_path}")

In [ ]:
# Create comprehensive results figure
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

fig.suptitle('Comprehensive Microcirculation Analysis Results', fontsize=16, fontweight='bold')

# 1. Patient summary table (as heatmap)
ax = fig.add_subplot(gs[0, :])
patient_summary = df_clinical.set_index('patient')[['mean_svd', 'mean_tvd', 'pct_continuous_flow']]
sns.heatmap(patient_summary.T, annot=True, fmt='.1f', cmap='RdYlGn', ax=ax, cbar_kws={'label': 'Value'})
ax.set_title('Patient Metrics Summary')

# 2-4. Distribution plots
ax = fig.add_subplot(gs[1, 0])
ax.hist(df_annotations['svd'], bins=15, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel('SVD')
ax.set_ylabel('Frequency')
ax.set_title('SVD Distribution')
ax.grid(True, alpha=0.3)

ax = fig.add_subplot(gs[1, 1])
ax.hist(df_annotations['tvd'], bins=15, color='coral', alpha=0.7, edgecolor='black')
ax.set_xlabel('TVD')
ax.set_ylabel('Frequency')
ax.set_title('TVD Distribution')
ax.grid(True, alpha=0.3)

ax = fig.add_subplot(gs[1, 2])
total_by_cat = [df_annotations['count_cat0'].sum(), df_annotations['count_cat1'].sum(),
                df_annotations['count_cat2'].sum(), df_annotations['count_cat3'].sum()]
colors_pie = ['red', 'orange', 'yellow', 'green']
ax.pie(total_by_cat, labels=['No Flow', 'Intermittent', 'Sluggish', 'Continuous'],
      colors=colors_pie, autopct='%1.1f%%', startangle=90)
ax.set_title('Vessel Flow Classification')

# 5-7. Clinical comparison
ax = fig.add_subplot(gs[2, 0])
class_order = ['Good', 'Moderate', 'Poor']
df_clinical_sorted = df_clinical.copy()
df_clinical_sorted['clinical_classification'] = pd.Categorical(df_clinical_sorted['clinical_classification'],
                                                               categories=class_order, ordered=True)
df_clinical_sorted = df_clinical_sorted.sort_values('clinical_classification')
colors = {'Good': 'green', 'Moderate': 'orange', 'Poor': 'red'}
color_list = [colors[c] for c in df_clinical_sorted['clinical_classification']]
ax.bar(range(len(df_clinical_sorted)), df_clinical_sorted['mean_svd'], color=color_list, alpha=0.7)
ax.set_ylabel('Mean SVD')
ax.set_xlabel('Patient')
ax.set_title('Baseline SVD by Prognosis')
ax.grid(True, alpha=0.3, axis='y')

ax = fig.add_subplot(gs[2, 1])
ax.bar(range(len(df_clinical_sorted)), df_clinical_sorted['pct_continuous_flow'], color=color_list, alpha=0.7)
ax.set_ylabel('% Continuous Flow')
ax.set_xlabel('Patient')
ax.set_title('Perfusion Quality by Prognosis')
ax.grid(True, alpha=0.3, axis='y')

ax = fig.add_subplot(gs[2, 2])
for classification in class_order:
    class_data = df_clinical[df_clinical['clinical_classification'] == classification]
    count = len(class_data)
    ax.barh(classification, count, color=colors[classification], alpha=0.7)
ax.set_xlabel('Number of Patients')
ax.set_title('Prognosis Distribution')
ax.grid(True, alpha=0.3, axis='x')

plt.savefig('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/thesis_summary_figure.png', 
           dpi=300, bbox_inches='tight')
plt.show()

print("\nThesis summary figure saved")

In [ ]:
# Comprehensive statistics
summary_stats = f"""

DATASET OVERVIEW
================
Total patients analyzed: {df_annotations['patient'].nunique()}
Patients: {', '.join(sorted(df_annotations['patient'].unique()))}
Total measurement records: {len(df_annotations)}
Mean records per patient: {len(df_annotations) / df_annotations['patient'].nunique():.1f}

Measurement timeline:
- Days recorded: Day {df_annotations['day_num'].min()} to Day {df_annotations['day_num'].max()}
- Maximum follow-up: {df_annotations['day_num'].max() - df_annotations['day_num'].min()} days

VESSEL METRICS SUMMARY (All measurements)
==========================================
Small Vessel Density (SVD):
  Mean: {df_annotations['svd'].mean():.2f}
  Std:  {df_annotations['svd'].std():.2f}
  Range: {df_annotations['svd'].min():.2f} - {df_annotations['svd'].max():.2f}

Total Vessel Density (TVD):
  Mean: {df_annotations['tvd'].mean():.2f}
  Std:  {df_annotations['tvd'].std():.2f}
  Range: {df_annotations['tvd'].min():.2f} - {df_annotations['tvd'].max():.2f}

Total Vessel Count:
  Mean: {df_annotations['total_vessels'].mean():.0f}
  Std:  {df_annotations['total_vessels'].std():.0f}
  Range: {df_annotations['total_vessels'].min():.0f} - {df_annotations['total_vessels'].max():.0f}

Vessel Flow Classification:
  No flow (Cat 0): {df_annotations['count_cat0'].sum():.0f} ({100*df_annotations['count_cat0'].sum()/df_annotations['total_vessels'].sum():.1f}%)
  Intermittent (Cat 1): {df_annotations['count_cat1'].sum():.0f} ({100*df_annotations['count_cat1'].sum()/df_annotations['total_vessels'].sum():.1f}%)
  Sluggish (Cat 2): {df_annotations['count_cat2'].sum():.0f} ({100*df_annotations['count_cat2'].sum()/df_annotations['total_vessels'].sum():.1f}%)
  Continuous (Cat 3): {df_annotations['count_cat3'].sum():.0f} ({100*df_annotations['count_cat3'].sum()/df_annotations['total_vessels'].sum():.1f}%)

CLINICAL CLASSIFICATION OUTCOMES
=================================
"""

for cls in ['Good', 'Moderate', 'Poor']:
    class_data = df_clinical[df_clinical['clinical_classification'] == cls]
    if len(class_data) > 0:
        summary_stats += f"\n{cls} Prognosis: {len(class_data)} patients\n"
        summary_stats += f"  Avg SVD: {class_data['mean_svd'].mean():.2f}\n"
        summary_stats += f"  Avg Continuous Flow: {class_data['pct_continuous_flow'].mean():.1f}%\n"
        for patient in class_data['patient'].values:
            summary_stats += f"    - {patient}\n"

summary_stats += f"""
INDIVIDUAL VESSEL VELOCITY ANALYSIS
====================================
Total vessels measured: {len([col for col in df_annotations.columns if col.startswith('velocity_')])} per record
Mean velocity across all measurements: {df_annotations[[col for col in df_annotations.columns if col.startswith('velocity_')]].values.flatten()[df_annotations[[col for col in df_annotations.columns if col.startswith('velocity_')]].values.flatten() > 0].mean():.0f}

KEY FINDINGS & INSIGHTS
=======================
1. Heterogeneous microvascular perfusion characterized by variable SVD and TVD
2. Classification into Good, Moderate, and Poor prognosis groups based on vessel density and perfusion quality
3. Temporal dynamics show patient-specific trends in vessel density over measurement period
4. Strong correlation between SVD and continuous flow percentage predicts better microvascular function
5. Individual vessel velocities vary significantly, suggesting complex hemodynamic interactions

RECOMMENDATIONS FOR FURTHER ANALYSIS
====================================
- Correlate clinical outcomes with temporal trends in vessel density
- Investigate patient-specific factors influencing SVD changes
- Develop predictive models for prognosis based on baseline metrics
- Compare results with clinical parameters (e.g., shock status, ICU outcomes)
"""

print(summary_stats)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Load all processed data
annotations_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/annotations_processed.csv')
df_annotations = pd.read_csv(annotations_path)

clinical_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/clinical_outcomes.csv')
df_clinical = pd.read_csv(clinical_path)

spacetime_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/spatiotemporal_features.csv')
if spacetime_path.exists():
    df_spacetime = pd.read_csv(spacetime_path)

print("=" * 100)
print("MICROCIRCULATION ANALYSIS - THESIS SUMMARY REPORT")
print("=" * 100)

# Notebook 09: Results Summary for Thesis

Comprehensive summary of all analysis findings with publication-ready visualizations and statistics.

# 09 - Resultados Finales para la Tesis

## Objetivo
Exportar:
- Gráficos
- Tablas resumen
- Resultados clave

## Formato
Se generan imágenes para la tesis y tablas en LaTeX o CSV


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(time, velocity)
plt.title("Velocidad de eritrocitos a lo largo del tiempo")
plt.xlabel("Tiempo (s)")
plt.ylabel("Velocidad (μm/s)")
plt.savefig("figures/velocity_curve.png")
